<a href="https://colab.research.google.com/github/Samyadel123/DEPI_Final/blob/main/Copy_of_DEPI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
grassknoted_asl_alphabet_path = kagglehub.dataset_download('grassknoted/asl-alphabet')

print('Data source import complete.')


Using Colab cache for faster access to the 'asl-alphabet' dataset.
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Streaming output truncated to the last 5000 lines.
/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/nothing/nothing2986.jpg
/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/nothing/nothing2747.jpg
/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/nothing/nothing941.jpg
/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/nothing/nothing2463.jpg
/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/nothing/nothing1971.jpg
/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/nothing/nothing2623.jpg
/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/nothing/nothing843.jpg
/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/nothing/nothing75.jpg
/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/nothing/nothing74.jpg
/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/nothing/nothing428.jpg
/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/n

In [ ]:
train_dir = '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
test_dir = '/kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test'

classes = sorted(os.listdir(train_dir))
print(f"Classes: {classes} ...")

Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space'] ...


In [ ]:
# %config Completer.use_jedi = False

to track our experments we well use mlflow

In [ ]:
#!pip install keras==3.0.2

In [ ]:
%pip install -q dagshub 'mlflow>=2,<3'


In [ ]:
import dagshub
dagshub.init(repo_owner='samyadel604', repo_name='MLflow-integration', mlflow=True)


Accessing as samyadel604

Initialized MLflow to track repo "samyadel604/MLflow-integration"

Repository samyadel604/MLflow-integration initialized!

In [ ]:
import mlflow
mlflow.autolog()

#mlflow.set_tracking_uri('https://dagshub.com/samyadel604/DEPI_MLFLOW.mlflow')

2025/11/24 22:40:23 WARNING mlflow.spark: With Pyspark >= 3.2, PYSPARK_PIN_THREAD environment variable must be set to false for Spark datasource autologging to work.
2025/11/24 22:40:23 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.


# mediapipe and mlp

In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled 

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import mediapipe as mp
import pickle
import tensorflow as tf
import shutil
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(
2025/11/24 22:40:39 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/11/24 22:40:39 WARNING mlflow.utils.autologging_utils: MLflow keras autologging is known to be compatible with 3.0.2 <= keras <= 3.9.2, but the installed version is 3.10.0. If you encounter errors during autologging, try upgrading / downgrading keras to a compatible version, or try upgrading MLflow.
2025/11/24 22:40:39 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/11/24 22:40:39 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


In [ ]:
mp_hands = mp.solutions.hands
hands_static = mp_hands.Hands(static_image_mode=True, # For reading the photos
                              max_num_hands=1,    # 1 hand per photo
                              min_detection_confidence=0.5     # Confidence level per photo
                             )

In [ ]:
# Extract Landmarks Function
def extract_landmarks(image_path):
    """Extract 21 hand landmarks (63 features: x, y, z for each point)"""
    # read the image in numpy array with BGR format
    image = cv2.imread(image_path)
    # Check if the image empty
    if image is None:
        return None
    # Converts to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Process it with the object of Media pipe
    results = hands_static.process(image_rgb)
    # Checks if any hand detected
    if results.multi_hand_landmarks:
        # get the first hand detected
        landmarks = results.multi_hand_landmarks[0]
        # Intiallize Corrdanate list
        coords = []
        # Extract x, y, z coordinates
        for lm in landmarks.landmark:
            coords.extend([lm.x, lm.y, lm.z])
        # Converts the coordinats into numpy array and return it
        return np.array(coords)
    return None

In [ ]:
# X array for all 63 corridnaate per hand
X_data = []
# For classes
y_data = []
# Define the classes
classes = sorted(os.listdir(train_dir))
print(f"Found {len(classes)} classes of {classes}")
# Iterate through each class image in the cropped images
for class_name in classes:
    class_path = os.path.join(train_dir, class_name)
    # skips the hidden files
    if not os.path.isdir(class_path):
        continue
    # Starts preprocessing images
    print(f"Processing class: {class_name}")
    files = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
    # COunt how many extracted correctly
    success_count = 0
    # Loops through each image and extract it
    for file in tqdm(files, desc=f"Extracting {class_name}", leave=False):
        img_path = os.path.join(class_path, file)
        landmarks = extract_landmarks(img_path)
        # If it extracted succecfully append into x and y
        if landmarks is not None:
            X_data.append(landmarks)
            y_data.append(class_name)
            success_count += 1

    print(f"Succisfully extracted {success_count} from {len(files)} samples")

print(f"\nTotal samples extracted: {len(X_data)}")

Found 29 classes of ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
Processing class: A


Succisfully extracted 2187 from 3000 samples
Processing class: B


Succisfully extracted 2207 from 3000 samples
Processing class: C


Succisfully extracted 1988 from 3000 samples
Processing class: D


Succisfully extracted 2463 from 3000 samples
Processing class: E


...

Succisfully extracted 2585 from 3000 samples
Processing class: Z


Succisfully extracted 2351 from 3000 samples
Processing class: del


Succisfully extracted 1701 from 3000 samples
Processing class: nothing


Succisfully extracted 3 from 3000 samples
Processing class: space


Succisfully extracted 1625 from 3000 samples

Total samples extracted: 63676


In [ ]:
# Remove "nothing" class samples (only has 3 samples)
print(f"Before filtering: {len(X_data)} samples")

# Find indices where class is NOT "nothing"
keep_indices = [i for i, label in enumerate(y_data) if label.lower() != 'nothing']

# Filter both X and y
X_data_filtered = [X_data[i] for i in keep_indices]
y_data_filtered = [y_data[i] for i in keep_indices]

# Replace original data
X_data = X_data_filtered
y_data = y_data_filtered

print(f"After filtering: {len(X_data)} samples")
print(f"Removed 'nothing' class")

# Check remaining classes
unique_classes = sorted(set(y_data))
print(f"Training on {len(unique_classes)} classes: {unique_classes}")

Before filtering: 63676 samples
After filtering: 63673 samples
Removed 'nothing' class
Training on 28 classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'space']


In [ ]:
# Remove "nothing" class samples (only has 3 samples)
print(f"Before filtering: {len(X_data)} samples")

# Find indices where class is NOT "nothing"
keep_indices = [i for i, label in enumerate(y_data) if label.lower() != 'nothing']

# Filter both X and y
X_data_filtered = [X_data[i] for i in keep_indices]
y_data_filtered = [y_data[i] for i in keep_indices]

# Replace original data
X_data = X_data_filtered
y_data = y_data_filtered

print(f"After filtering: {len(X_data)} samples")
print(f"Removed 'nothing' class")

# Check remaining classes
unique_classes = sorted(set(y_data))
print(f"Training on {len(unique_classes)} classes: {unique_classes}")

Before filtering: 63673 samples
After filtering: 63673 samples
Removed 'nothing' class
Training on 28 classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'space']


In [ ]:
# Transform X and y to array for splitting
X = np.array(X_data)
y = np.array(y_data)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y_categorical, test_size=0.15, random_state=42, stratify=y_encoded
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Number of classes: {len(label_encoder.classes_)}")
print(f"Feature dimension: {X_train.shape[1]} (21 landmarks × 3 coordinates)")


Training samples: 54122
Validation samples: 9551
Number of classes: 28
Feature dimension: 63 (21 landmarks × 3 coordinates)


In [ ]:
# Transform X and y to array for splitting
X = np.array(X_data)
y = np.array(y_data)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y_categorical, test_size=0.15, random_state=42, stratify=y_encoded
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Number of classes: {len(label_encoder.classes_)}")
print(f"Feature dimension: {X_train.shape[1]} (21 landmarks × 3 coordinates)")

Training samples: 54122
Validation samples: 9551
Number of classes: 28
Feature dimension: 63 (21 landmarks × 3 coordinates)


In [ ]:
np.save('X_train.npy', X_train)
np.save('X_val.npy', X_val)
np.save('y_train.npy', y_train)
np.save('y_val.npy', y_val)

In [ ]:
# Define number of classes
#num_classes = len(label_encoder.classes_)
# Build a sequential model
model = Sequential([
    # 256 Dense layer
    Dense(256, activation='relu', input_shape=(63,)),
    # Normalize activations to stablize training
    BatchNormalization(),
    # Drop out layer to make the model learn more
    Dropout(0.4),
    # Second Dense layer and the same as befor
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    # Third dense layer with the same as before
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    # Output layer
    Dense(28, activation='softmax')
])
# COmpile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
# GEt number of parameters per layer
model.summary()

# Callbacks
callbacks = [
    # If the model didn't improve for 10 epochs stop the training to prevent overfitting
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    # Save the best model in training
    ModelCheckpoint('asl_landmarks_best.keras', save_best_only=True, monitor='val_accuracy'),
    # Reduce learning rate by .5 if the loss stops improving per 5 epochs
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)
]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 28)             │         1,820 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,148 (238.86 KB)

 Trainable params: 60,252 (235.36 KB)

 Non-trainable params: 896 (3.50 KB)

In [ ]:
# Train Model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=128,
    callbacks=callbacks,
    verbose=1
)

2025/11/24 23:27:16 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9d4dac0362be44ef810bc2562f8fb9be', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/100


2025/11/24 23:27:24 INFO mlflow.bedrock: Enabled auto-tracing for Bedrock. Note that MLflow can only trace boto3 service clients that are created after this call. If you have already created one, please recreate the client by calling `boto3.client`.
2025/11/24 23:27:24 INFO mlflow.tracking.fluent: Autologging successfully enabled for boto3.


423/423 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5678 - loss: 1.5986

423/423 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.5683 - loss: 1.5968 - val_accuracy: 0.7712 - val_loss: 0.7282 - learning_rate: 0.0010
Epoch 2/100
421/423 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9341 - loss: 0.2537

...

423/423 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9797 - loss: 0.0736 - val_accuracy: 0.9909 - val_loss: 0.0336 - learning_rate: 6.2500e-05
Epoch 59/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9803 - loss: 0.0704 - val_accuracy: 0.9904 - val_loss: 0.0342 - learning_rate: 6.2500e-05
Epoch 60/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9802 - loss: 0.0705 - val_accuracy: 0.9910 - val_loss: 0.0350 - learning_rate: 6.2500e-05
Epoch 61/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9801 - loss: 0.0695 - val_accuracy: 0.9910 - val_loss: 0.0338 - learning_rate: 6.2500e-05
Epoch 62/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9799 - loss: 0.0739 - val_accuracy: 0.9893 - val_loss: 0.0373 - learning_rate: 6.2500e-05
Epoch 63/100
415/423 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9797 - loss: 0.0677

423/423 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9797 - loss: 0.0677 - val_accuracy: 0.9913 - val_loss: 0.0333 - learning_rate: 6.2500e-05
Epoch 64/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9797 - loss: 0.0699 - val_accuracy: 0.9892 - val_loss: 0.0367 - learning_rate: 6.2500e-05
Epoch 65/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9798 - loss: 0.0695 - val_accuracy: 0.9912 - val_loss: 0.0335 - learning_rate: 6.2500e-05
Epoch 66/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9797 - loss: 0.0692 - val_accuracy: 0.9903 - val_loss: 0.0351 - learning_rate: 6.2500e-05
Epoch 67/100
411/423 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9804 - loss: 0.0693

423/423 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9804 - loss: 0.0693 - val_accuracy: 0.9905 - val_loss: 0.0332 - learning_rate: 6.2500e-05
Epoch 68/100
413/423 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9791 - loss: 0.0712

423/423 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9791 - loss: 0.0712 - val_accuracy: 0.9905 - val_loss: 0.0332 - learning_rate: 6.2500e-05
Epoch 69/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9811 - loss: 0.0656 - val_accuracy: 0.9909 - val_loss: 0.0333 - learning_rate: 3.1250e-05
Epoch 70/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9818 - loss: 0.0646 - val_accuracy: 0.9912 - val_loss: 0.0333 - learning_rate: 3.1250e-05
Epoch 71/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9809 - loss: 0.0637 - val_accuracy: 0.9907 - val_loss: 0.0336 - learning_rate: 3.1250e-05
Epoch 72/100
412/423 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9797 - loss: 0.0684

423/423 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9798 - loss: 0.0684 - val_accuracy: 0.9913 - val_loss: 0.0328 - learning_rate: 3.1250e-05
Epoch 73/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9810 - loss: 0.0671 - val_accuracy: 0.9909 - val_loss: 0.0336 - learning_rate: 3.1250e-05
Epoch 74/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9809 - loss: 0.0671 - val_accuracy: 0.9914 - val_loss: 0.0332 - learning_rate: 3.1250e-05
Epoch 75/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9801 - loss: 0.0690 - val_accuracy: 0.9913 - val_loss: 0.0334 - learning_rate: 3.1250e-05
Epoch 76/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9812 - loss: 0.0672 - val_accuracy: 0.9917 - val_loss: 0.0329 - learning_rate: 3.1250e-05
Epoch 77/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9802 - loss: 0.0702 - val_accuracy: 0.9913 - val_loss: 0.0334 - learning_rate: 3.1250e-05
Epoch 78/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy

In [ ]:
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"\nFinal Validation Accuracy= {val_accuracy*100:.2f}%")
print(f"Final Validation Loss= {val_loss:.4f}")